In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage

import os
import json
import pandas as pd
import openai
import faiss

from llama_index.core import Document
from openai import OpenAI
from dotenv import load_dotenv 
from datetime import datetime, timedelta


In [ ]:
# load data.
# segregate the rows
# to vector db (faiss or other)
# search (faiss, qdrant)
# chunks
# retrieve
# mini prompt
# generate
# include function calling
# advanced prompt

In [8]:
# Load your Excel file
df = pd.read_excel("data.xlsx")  # Ensure this is the correct path

# Convert each row into a separate Document

documents = [Document(text=str(row)) for row in df.itertuples(index=False, name=None)]

In [72]:
documents_text = [doc.text for doc in documents]
documents_text
user_question = "Какие фанари есть для Cruise B7 и их цены?"


In [34]:
response = openai.embeddings.create(
    model="text-embedding-3-large",
    input=documents_text,
)

embeddings = [item.embedding for item in response.data]
embeddings = np.array(embeddings, dtype='float32')
embeddings

array([[-4.7778763e-02,  2.2870261e-02, -1.9782927e-02, ...,
        -3.0783432e-02, -1.2933837e-02, -1.0311101e-02],
       [-4.0353980e-02,  1.1482712e-02, -1.6025668e-02, ...,
        -3.3649202e-02, -8.1851529e-03, -7.8209331e-03],
       [-2.3129556e-02, -2.3129556e-02, -1.0510755e-02, ...,
        -4.5197718e-02, -2.5724077e-03,  1.2733049e-03],
       ...,
       [-1.5606306e-02, -3.5781652e-02, -8.8559594e-03, ...,
        -2.3051642e-02,  5.4188562e-03,  3.3168992e-05],
       [-3.3165172e-02, -3.1921477e-03, -1.2718122e-02, ...,
        -2.8305754e-02,  4.1708807e-03, -3.5241600e-02],
       [-2.3747798e-02, -3.3621960e-03, -9.6470853e-03, ...,
        -2.3923598e-02,  3.7430984e-03, -2.5124906e-02]], dtype=float32)

In [39]:
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)


In [ ]:

dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # IP = inner product
index.add(embeddings)

# faiss.write_index(index, "my_index.index")

In [41]:
query_embedding = openai.embeddings.create(
    model="text-embedding-3-large",
    input=["Сколько будет стоить задний фонарь на Пассат Б6?"]
).data[0].embedding

In [42]:
query_embedding = np.array(query_embedding, dtype='float32')
query_embedding = query_embedding / np.linalg.norm(query_embedding)  # normalize

In [43]:
D, I = index.search(query_embedding.reshape(1, -1), k=3)
print("Top-3 индексы:", I)
print("Top-3 сходства:", D)


Top-3 индексы: [[3 2 4]]
Top-3 сходства: [[0.534588   0.5256282  0.47542885]]


In [62]:

# print("\n🎯 Top-k results:")
# for rank, idx in enumerate(I[0]):
#     print(f"{rank + 1}. (Score: {D[0][rank]:.4f}) — {documents_text[idx]}")
    
    

chunks = [documents[i] for i in I[0]]
context = "\n".join(f"{i+1}. {chunk.text}" for i, chunk in enumerate(chunks))
print(type(context))

<class 'str'>


In [140]:
"""Find the relevant chunks"""
client = OpenAI()

    

messages_new = [
    {"role": "system", "content": instructions_text},    
    # 🔽 Здесь вставляются найденные документы/чанки из RAG
    {"role": "system", "name": "retrieval_context", "content":context
    },

    {"role": "user", "content": user_question}
]

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages_new,
)

output_new = completion.choices[0].message.content
print(output_new)

Для Cruiser B7 у нас есть задний фонарь аналогичный Depo, цена которого составляет 3600 ₽.


In [ ]:
function_description = [
    {
        "name":"send_invoice",
        "description":"Отправить клиенту счет на оплату товара, чтобы завершить сделку. Выполняется только тогда когда получены все данные.",
        "parameters":{
            "type":"object",
            "properties":{
                "product_name":{
                    "type":"string",
                    "description":"Наименование товара, запчасти. Например: Оригинальный задний правый фанарь для Cruiser B7 (2010–2015).",
                },
                "amount":{
                    "type":"integer",
                    "description":"Количество товара. Например: 3"
                },
                "total_price":{
                    "type":"number",
                    "description": "Общая стоимость заказа, включая стоимость всех единиц товара, а также дополнительные сборы, такие как доставка, налоги или скидки, если применимо. Например: 3500.21"
                },
                "payment_method":{
                    "type":"string",
                    "description":"Способ оплаты"
                },
                "customer_name":{
                    "type":"string",
                    "description":"Имя покупателя"
                },
                "customer_contact":{
                    "type":"string",
                    "description": "Контакт покупателя. Мобильный номер либо любой другой контакт. Например: +7 700 787 88 77"
                    }
                },
        "required":["product_name", "amount", "total_price","payment_method","customer_name", "customer_contact"]
        }
    },
    {
        "name": "handover_to_manager",
        "description": "Передать «тёплого» или «горячего» клиента живому менеджеру для завершения продажи или уточнения нюансов. Уведомлять менеджера о заинтересованном клиенте.",
        "parameters": {
            "type": "object",
            "properties": {
                "customer_name": {
                    "type": "string",
                    "description": "Имя клиента"
                },
                "customer_contact": {
                    "type": "string",
                    "description": "Контактный номер телефона клиента или email"
                },
                "products": {
                    "type": "array",
                    "description": "Список товаров, которые интересуют клиента",
                    "items": {
                        "type": "object",
                        "properties": {
                            "product_name": {
                                "type": "string",
                                "description": "Наименование товара"
                            },
                            "product_price": {
                                "type": "number",
                                "description": "Цена товара"
                            },
                            "product_quantity": {
                                "type": "number",
                                "description": "Количество товара"
                            },
                            "product_type": {
                                "type": "string",
                                "description": "Тип товара (например, оригинал или аналог)"
                            }
                        },
                        "required": ["product_name", "product_price"]
                    }
                },
                "status": {
                    "type": "string",
                    "description": "Статус клиента, чем помочь или какая проблема. Например 'Нужна консультация по гарантии продукта'"
                },
                "preferred_contact_time": {
                    "type": "string",
                    "description": "Предпочтительное время для связи, например 'К вечеру после работы'"
                }
            }
        }
    }
    ]

In [197]:
def send_invoice(product_name, amount, total_price,payment_method,customer_name, customer_contact):
    info_recepie = {
        "product_name" : product_name,
        "amount" : amount,
        "total_price" : total_price,
        "payment_method" : payment_method,
        "customer_name" : customer_name,
        "customer_contact" : customer_contact
    }
    
    return json.dumps(info_recepie, ensure_ascii=False, indent=2)

def handover_to_manager(customer_name, customer_contact, products, status, preferred_contact_time):
    info_to_manager = {
        "customer_name": customer_name,
        "customer_contact": customer_contact,
        "products": products,
        "status":status,
        "prefered_contact_time": preferred_contact_time
    }
    
    return json.dumps(info_to_manager, ensure_ascii=False, indent=2)

In [230]:

instructions_text = """
Ты **продавец-ассистент** для интернет-магазина автозапчастей для немецких автомобилей. 
Твоя главная роль — помочь клиенту с выбором запчастей и всячески способствовать тому, чтобы клиент купил продукт. 
Ты гений маркетинга и продаж, что даже Волк с Уолл-стрит позавидует твоим навыкам.

Тебе нужно обдумать и отвечать на вопросы. Чтобы ответить на вопросы, тебе могут пригодиться следующие данные:

1. Использовать retrieval context, если ты думаешь, что клиент спрашивает что-то связанное с продуктом.  
   В документах данные представлены в следующем порядке: ID; Название детали; Совместимость моделей; Оригинал; Цена (₽); Артикул.  
   Ты должен выводить только релевантные данные и избегать добавления нерелевантной информации.

2. **История чата**: Пожалуйста следи за историей чата и размышляй как ответить именно в контексте истории чата. Если будут вопросы не связанные с retrieval context то лучше прислушайся к истории чата. 
Важно: если клиент говорит "второй", "предыдущий", "другой", "второй из предложенных" — ты должен использовать ИСТОРИЮ ЧАТА, а не просто порядок retrieval context.

Вызовы функции:  
У тебя есть несколько функций, которые нужно использовать в нужные моменты.

1. **send_invoice**:  
   При вызове этой функции, например, send_invoice, пожалуйста, спроси у клиента по типу 'Готовы ли вы приобрести запчасть?', если ответ положительный то ты должен получить данные клиента: customer_name, customer_contact, payment_method  
   Ты должен четко понимать когда использывать эту функцию.  
   Например, если клиент уже доволен продуктом, ты можешь начать спрашивать его личные данные, такие как имя (customer_name), телефон (customer_contact) и способ оплаты (payment_method). 
   Ты не можешь вызывать функцию send_invoice, пока не получил все необходимые данные от клиента: имя, контакт, способ оплаты. Если этих данных нет — ты должен их запросить. Даже если клиент сказал, что хочет купить.

2. **handover_to_manager**:  

Ты можешь использовать эту функцию, если:
- ты **точно не знаешь ответ на вопрос клиента**, 
- или **не уверен** в каком-либо аспекте (например, редкая модель машины, особые условия заказа, необычные запросы),
- и ты считаешь, что клиенту будет полезна консультация менеджера.

Твоя задача — **предложить** клиенту связаться с менеджером **только если** ты полностью уверен, что это нужно, и ты **получил все необходимые данные**.

Перед вызовом функции ты **обязан**:
- Уточнить, **хочет ли клиент поговорить с менеджером**. Например: "Вы хотите, чтобы я передал ваш запрос менеджеру?".
- Спросить и получить следующие данные:
  - **Имя клиента (customer_name)**
  - **Контакт клиента (customer_contact)** — телефон или мессенджер
  - **Какие детали интересуют (products)** — ты можешь заполнить это из контекста, если ранее обсуждали детали
  - **Статус клиента (status)** — например: "Статус клиента, чем помочь или какая проблема. Например 'Нужна консультация по гарантии продукта'"
  - **Предпочтительное время для связи (preferred_contact_time)** — обязательно спроси: "В какое время вам будет удобно, чтобы менеджер связался с вами?"

💡 Если хотя бы один из этих пунктов отсутствует — **не вызывай функцию**. Вместо этого скажи что-то вроде:  
"Чтобы передать ваш запрос менеджеру, мне нужно уточнить еще {недостающие поля}. Подскажите, пожалуйста."

Ты можешь вызывать `handover_to_manager` **только** если ты уверен, что:
1. Клиент согласен на контакт с менеджером,
2. Все обязательные поля собраны и заполнены.



Перед тем как запускать функцию тебе нужно проверить все ли данные в порядке, получил ли ты личные данные от клиента. 
- Если уверен что данные получены то пользуйся функциями.
- Если есть сомнения то обязательно спроси в нужное время.

**Структура ответов**
1. Если ты нашёл несколько релевантных запчастей (2 или более), ты обязан вывести их в виде списка с чёткой нумерацией:

Пример:
🔧 Вариант 1:
Название: ...
Цена: ...
Артикул: ...

🔧 Вариант 2:
Название: ...
Цена: ...
Артикул: ...


"""


dialog_history = [
    {"role": "system", "content": instructions_text}
]


def chat_step(user_question, context):
    dialog_history.append({"role": "system", "name": "retrieval_context", "content": context})
    dialog_history.append({"role": "user", "content": user_question})

    completion = client.chat.completions.create(
        model="gpt-4.1-mini-2025-04-14",
        messages=dialog_history,
        functions=function_description,
        function_call="auto"
    )

    message = completion.choices[0].message

    if message.function_call:
        name = message.function_call.name
        arguments = json.loads(message.function_call.arguments)

        if name == "send_invoice":
            result = send_invoice(**arguments)
            dialog_history.append({"role": "function", "name": name, "content": result})
            return f"Счет успешно выставлен. Подробности:\n{result}"

        elif name == "handover_to_manager":
            result = handover_to_manager(**arguments)  
            dialog_history.append({"role": "function", "name": name, "content": result})
            return f"Клиент передан менеджеру. Подробности:\n{result}"

        else:
            return f"Неизвестная функция: {name}"

    elif message.content:
        dialog_history.append({"role": "assistant", "content": message.content})
        return message.content


In [231]:
dialog_history

[{'role': 'system',
  'content': '\nТы **продавец-ассистент** для интернет-магазина автозапчастей для немецких автомобилей. \nТвоя главная роль — помочь клиенту с выбором запчастей и всячески способствовать тому, чтобы клиент купил продукт. \nТы гений маркетинга и продаж, что даже Волк с Уолл-стрит позавидует твоим навыкам.\n\nТебе нужно обдумать и отвечать на вопросы. Чтобы ответить на вопросы, тебе могут пригодиться следующие данные:\n\n1. Использовать retrieval context, если ты думаешь, что клиент спрашивает что-то связанное с продуктом.  \n   В документах данные представлены в следующем порядке: ID; Название детали; Совместимость моделей; Оригинал; Цена (₽); Артикул.  \n   Ты должен выводить только релевантные данные и избегать добавления нерелевантной информации.\n\n2. **История чата**: Пожалуйста следи за историей чата и размышляй как ответить именно в контексте истории чата. Если будут вопросы не связанные с retrieval context то лучше прислушайся к истории чата. \nВажно: если кл

In [228]:
def retrieve_context(user_question, index, documents):

    query_embedding = openai.embeddings.create(
        model="text-embedding-3-large",
        input=[user_question]
    ).data[0].embedding

    query_embedding = np.array(query_embedding, dtype='float32')
    query_embedding = query_embedding / np.linalg.norm(query_embedding) 
    
    D, I = index.search(query_embedding.reshape(1, -1), k=3)

    chunks = [documents[i] for i in I[0]]
    context = "\n".join(f"{i+1}. {chunk.text}" for i, chunk in enumerate(chunks))
    # context = "\n".join(f"{chunk.text}" for i, chunk in enumerate(chunks))
    
    return context


In [234]:
questions = [
    "Привет, ты что вообще делаешь?",
    "Я хочу купить задний фонарь для Cruiser B7. Какие варианты есть?",
    "Можно подробнее про второй вариант который ты мне показал?",
    "Отлично, этот мне подходит, хочу купить",
    "1. Томас, 2. 8 900 80 80 7475, 3. Хочу оплатить картой онлайн. Но я хочу еще узнать про гарантию и получить чек физический, как я могу это получить?",
    "Да, свяжи меня с ним",
    "Мне норм в любое время. Хочу просто спросить про гарантию у живого человека. А запчасть та которую я хотел",
    "А ладно, мне не нужна консультация менеджера. Я готовь прямо сейчас покупать"
    ]

for i, question in enumerate(questions):    
    context = retrieve_context(question, index, documents)
    reply = chat_step(question, context)
    print(f"\n==================================\nВопрос {i+1}: {question}")
    print(reply)
    
    



Вопрос 1: Привет, ты что вообще делаешь?
Привет! Я помогу тебе с подбором и покупкой автозапчастей для немецких автомобилей. Если хочешь найти нужную деталь или получить совет, просто скажи, и я подберу лучшие варианты для тебя. Какую запчасть ищешь?

Вопрос 2: Я хочу купить задний фонарь для Cruiser B7. Какие варианты есть?
Вот два варианта заднего фонаря для Cruiser B7:

🔧 Вариант 1:  
Название: Задний фонарь правый  
Оригинал: Да  
Цена: 5900 ₽  
Артикул: LGT-201-O  

🔧 Вариант 2:  
Название: Задний фонарь (аналог Depo)  
Оригинал: Нет  
Цена: 3600 ₽  
Артикул: LGT-201-D  

Если нужно, помогу выбрать или оформить заказ.

Вопрос 3: Можно подробнее про второй вариант который ты мне показал?
Второй вариант заднего фонаря для Cruiser B7 — это "Задний фонарь (аналог Depo)":

- Цена: 3600 ₽  
- Артикул: LGT-201-D  
- Совместимость: Cruiser B7  
- Оригинал: Нет, это качественный аналог от Depo  
- Отличается более доступной ценой при достойном качестве

Если хотите, могу помочь с оформлен

In [ ]:
question = "Отлично, этот мне подходит, хочу купить" 
context = retrieve_context(question, index, documents)
reply = chat_step(question, context)
print(reply)


Извините за путаницу! Второй фонарь — это именно оригинальный правый задний фонарь для Cruiser B7 (2010–2015) с артикулом LGT-201-O и ценой 5900 ₽.

Первый — это аналог от Depo, который дешевле — 3600 ₽, без оригинальных гарантий.

Если хотите, могу снова подробно рассказать именно про оригинал или помочь с оформлением заказа. Что предпочитаете?


In [ ]:
question = "1. Томас, 2. 8 900 80 80 7475, 3. Хочу оплатить картой онлайн. Но я хочу еще узнать про гарантию и получить чек физический, как я могу это получить?" 
context = retrieve_context(question, index, documents)
reply = chat_step(question, context)
print(reply)



Максат, спасибо за информацию! По гарантии: на оригинальные запчасти, включая задний фонарь, распространяется официальная гарантия от производителя. В случае заводского брака или неисправности в течение гарантийного срока вы сможете обратиться к нам для замены или возврата.

Что касается физического чека — мы можем предоставить его вместе с доставкой вашего заказа.

Если хотите, могу сейчас сформировать для вас счет на оплату через Сбербанк по QR-коду и оформить заказ. Все вопросы по гарантии и чеку мы с вами учтем. Как вам такой вариант?


In [203]:
question = "Да, свяжи меня с ним" 
context = retrieve_context(question, index, documents)
reply = chat_step(question, context)
print(reply)


Максат, чтобы передать ваш запрос менеджеру, мне нужно уточнить некоторые детали:

1. Подтвердите, пожалуйста, ваше имя — Максат, верно?  
2. Ваш контакт — +7 870 796 77 877?  
3. Какие именно детали интересуют вас? Сейчас мы обсуждаем задний правый фонарь для Cruiser B7?  
4. Уточните, пожалуйста, ваш статус или чем менеджер должен помочь — например, нужна консультация по гарантии и оформлению чека?  
5. В какое время вам будет удобно, чтобы менеджер с вами связался?

Как только уточните эти моменты, я передам ваш запрос.


In [204]:
question = "Мне норм в любое время. Хочу просто спросить про гарантию у живого человека. А запчасть та которую я хотел" 
context = retrieve_context(question, index, documents)
reply = chat_step(question, context)
print(reply)


Клиент передан менеджеру. Подробности:
{
  "customer_name": "Максат",
  "customer_contact": "+7 870 796 77 877",
  "products": [
    {
      "product_name": "Задний фонарь правый для Cruiser B7",
      "product_price": 5900
    }
  ],
  "status": "Хочу уточнить вопросы по гарантии у менеджера",
  "prefered_contact_time": "В любое время"
}


In [205]:
question = "А ладно, мне не нужна консультация менеджера. Я готовь прямо сейчас покупать" 
context = retrieve_context(question, index, documents)
reply = chat_step(question, context)
print(reply)


Отлично, Максат! Для оформления заказа на оригинальный задний правый фонарь для Cruiser B7 с оплатой через QR-код Сбербанка, подтвердите, пожалуйста, ещё раз ваш способ оплаты — QR-код Сбербанка? И я сразу подготовлю счет.


In [ ]:
question = "Да, все верно" 
context = retrieve_context(question, index, documents)
reply = chat_step(question, context)
print(reply)


Счет успешно выставлен. Подробности:
{
  "product_name": "Задний фонарь правый для Cruiser B7",
  "amount": 1,
  "total_price": 5900,
  "payment_method": "QR-код Сбербанка",
  "customer_name": "Максат",
  "customer_contact": "+7 870 796 77 877"
}


None
